# TCC Augusto e Ícaro
## Modelo de automatização das Heurísticas de Nielsen para comentários em reviews de Apps

* Versão 0.1.0
* Bibliotecas utilizadas: pandas, numpy, texthero, ntlk e corpus do ntlk em português
* Dataset utilizado: dataset_v4.csv
* Data: 22/07/2020

### Objetivos principais:

* Detalhar e implementar pipeline de pre-processamento

* Implementar classificador baseado em ocorrência, sem foco na acurácia do modelo

In [1]:
import sys
!{sys.executable} -m pip install pandas
!{sys.executable} -m pip install numpy
!{sys.executable} -m pip install texthero 
!{sys.executable} -m pip install nltk 

In [3]:
import pandas as pd
import texthero as hero
import nltk
import numpy as np
# nltk.download()

## Pré-processamento

Nessa versão iremos testar apenas com textos em português, sem adicionar os pesos de sentimentos e heurísticas de Nielsen. 

### Carregamento dos dados

Queremos treinar o modelo para classificar apenas em usabilidade ou não.
Para isso, iremos utilizar apenas as colunas `is_usability` e `is_classified` do dataset.

In [2]:
cols = ['ID', 'Text', 'is_usability', 'is_classified']
df = pd.read_csv("reviews_v4.csv", index_col=False, usecols=cols)
df = df.rename(columns={'Text': 'text'})
df.head()

NameError: name 'pd' is not defined

In [66]:
df['is_usability'].value_counts()

False    755
True     175
Name: is_usability, dtype: int64

In [67]:
df['is_classified'].value_counts()

False    548
True     381
Name: is_classified, dtype: int64

### Limpando dados

O pipeline padrão remove dígitos, pontuação, remove diacritics, stopwords em inglês e whitespace

In [68]:
pipeline = hero.preprocessing.get_default_pipeline()
selected_functions_indexes = [0, 1, 2, 5, 6]
pipeline = [pipeline[i] for i in selected_functions_indexes]
df['text'] = hero.preprocessing.clean(df['text'])
df['text']

0      negocio de reconhecimento facialnao funciona n...
1      um lixo tentativas de econhecimento facial sem...
2      horrivel pior que fbi se fosse pra receberem a...
3           meio dificil fazer sem oculos mais deu certo
4         nao serve pra nada nao da para acessar pessimo
                             ...                        
925    como eu faco pra entrar na minha conta enem pe...
926    sim facilita muito em varios servicos e mais i...
927    nao consigo acessar meu auxilio como faco pra ...
928                                  seguro hiper seguro
929    otimo porem e bom implementar novas funcionali...
Name: text, Length: 930, dtype: object

### Stopwords

Removendo stopwords em português com o corpus em portugês do NTLK

In [69]:
from nltk.corpus import stopwords
pt_stopwords = stopwords.words('portuguese')
df['text'] = hero.remove_stopwords(df['text'], stopwords=pt_stopwords)
df['text'][0]

'negocio  reconhecimento facialnao funciona nao  pra ficar dia todo nisso nao arrumem   favor'

### Stemização

In [70]:
df['text'] = hero.stem(df['text'], language='portuguese')
df['text']
df['text'][0]

'negoci reconhec facialna funcion nao pra fic dia tod niss nao arrum favor'

### Tokenização

In [71]:
df['text'] = hero.tokenize(df['text'])
df.head()

,ID,text,is_usability,is_classified
0,gp:AOqpTOEAoU7yqbku6p11S8xAmTVWRkt0qC0lG4I-XH-...,"[negoci, reconhec, facialna, funcion, nao, pra...",True,True
1,gp:AOqpTOGoBVvBdtutEKaPnD7--OBC8nDGbSuRniM3Vbm...,"[lix, tentat, econhec, facial, exit, dur, depe...",False,True
2,gp:AOqpTOFUV6HfFseu9It6e2HTJy3CWzCuNgqCyelLRka...,"[horrivel, pior, fbi, pra, receb, algo, gent, ...",True,True
3,gp:AOqpTOG3mZqb8oLGMk-njqh5ycoCJ0EMNIVEghrRUzg...,"[mei, dificil, faz, ocul, deu, cert]",True,True
4,gp:AOqpTOE-kXsLJBlMyY8W7NWsdk_XrfVsYpOZSATrW5v...,"[nao, serv, pra, nad, nao, acess, pessim]",True,True


In [72]:
df['class_name'] = df.apply(lambda row: 'usability' if row['is_usability'] else 'not_usability', axis=1)

In [73]:
df = df.drop('is_usability', 1)
df = df.to_dict('records')
df[0]

{'ID': 'gp:AOqpTOEAoU7yqbku6p11S8xAmTVWRkt0qC0lG4I-XH-aKqUYoBvlXRVBPmOCEfu4c8d3x95IvEF06yqPaEDEwQ',
 'text': ['negoci',
  'reconhec',
  'facialna',
  'funcion',
  'nao',
  'pra',
  'fic',
  'dia',
  'tod',
  'niss',
  'nao',
  'arrum',
  'favor'],
 'is_classified': True,
 'class_name': 'usability'}

## NLP

Nessa parte, iremos treinar e utilizar o modelo

Função para aprendizado dos dados: 

In [74]:
def learning(training_data):
    corpus_words = {}
    for data in training_data: 
        class_name = data['class_name']
        frase = data['text']
        if class_name not in list(corpus_words.keys()):
            corpus_words[class_name] = {}
        for word in frase:
            if word not in list(corpus_words[class_name].keys()):
                corpus_words[class_name][word] = 1
            else:
                corpus_words[class_name][word] += 1
    return corpus_words

In [75]:
def remove_low_score(corpus_words):
    for word in list(corpus_words['usability'].keys()):
        if corpus_words['usability'][word] < 10:
            del corpus_words['usability'][word]

Função para calcular scores

In [76]:
def classificate(corpus, sentence):
    def calculate_class_score(corpus_words, sentence, class_name):
        score = 0 
        for word in sentence:
            if word in corpus_words[class_name]:
                score += corpus_words[class_name][word]
        return score
    classifications = []
    for class_name in corpus.keys():
        classifications.append({'class_name': class_name, 'score': calculate_class_score(corpus, sentence, class_name)})    
    return classifications

Função para normalizar os scores

In [77]:
def normalize_scores(classification):
    total_score = sum(score['score'] for score in classification['scores'])
    if total_score != 0:
        for score in classification['scores']:
            score['score'] = score['score']/total_score
    return classification

### Função principal

* Divide o dataset em classificados e não classificados.
* Divide os classificados para teste e treino: 20% teste e 80% treino.
* Calcula em 50 interações a média da acurácia do modelo.

In [78]:
usability_corpus = {
     'form': 100,
     'celul': 100,
     'vez': 110,
     'consegu': 120,
     'email': 120,
     'pod': 130,
     'ajud': 130,
     'opca': 130,
     'ped': 130,
     'entrar': 140,
     'fiz': 140,
     'fot': 140,
     'ser': 150,
     'aparec': 150,
     'prov': 150,
     'fac': 150,
     'diz': 160,
     'temp': 170,
     'tod': 170,
     'cnh': 170,
     'nad': 180,
     'vid': 190,
     'fic': 190,
     'cont': 200,
     'cpf': 210,
     'recuper': 220,
     'precis': 250,
     'acess': 260,
     'reconhec': 270,
     'erro': 280,
     'aplic': 290,
     'funcion': 290,
     'facial': 300,
     'app': 350,
     'tent': 370,
     'cadastr': 410,
     'consig': 430,
     'faz': 560,
     'senh': 570,
}

In [88]:
import numpy
import random

classified_df = []
unclassified_df = []

for data in df:
    classified_df.append(data) if data['is_classified'] else unclassified_df.append(data)

accuracies = []
iterations = 1
for i in range(iterations):
    train_dataset = []
    test_dataset = []
    random.shuffle(classified_df)
    train_dataset, test_dataset = np.split(classified_df, [int(len(classified_df)*0.1)])
        
    usa = [data for data in test_dataset if data['class_name']=='usability']
    not_usa = [data for data in test_dataset if data['class_name']=='not_usability']

#     corpus = learning(train_dataset)
    corpus['usability'] = tmp_corpus
    classifications = [{'test_data': test_data, 'scores': classificate(corpus, test_data['text'])} for test_data in test_dataset]
    classifications = [normalize_scores(classification) for classification in classifications]
    
    right_answers_usa = 0
    right_answers_no_usa = 0
    for classification in classifications:
        higher_score = max(classification['scores'], key=lambda x:x['score'])
        if classification['test_data']['class_name'] == higher_score['class_name']:
            if classification['test_data']['class_name'] == 'usability':
                right_answers_usa+=1
            else: 
                right_answers_no_usa+=1
        else: 
            print(classification)
    accuracy = right_answers/len(classifications)
    accuracies.append(accuracy)
    print(f'Iteration {i} - Accuracy {accuracy}')

print('-----------------')
print(f'Final accuracy {sum(accuracies)/len(accuracies)}')
print(f'Right answers usa: {right_answers_usa}  qtd {len(usa)}')
print(f'Right answers no_usa: {right_answers_no_usa} qtd {len(not_usa)}')
print('-----------------')

Iteration 0 - Accuracy 0.10174418604651163
-----------------
Final accuracy 0.10174418604651163
Right answers usa: 149  qtd 158
Right answers no_usa: 66 qtd 186
-----------------


In [80]:
print(corpus['usability'])

{'form': 100, 'celul': 100, 'vez': 110, 'consegu': 120, 'email': 120, 'pod': 130, 'ajud': 130, 'opca': 130, 'ped': 130, 'entrar': 140, 'fiz': 140, 'fot': 140, 'ser': 150, 'aparec': 150, 'prov': 150, 'fac': 150, 'diz': 160, 'temp': 170, 'tod': 170, 'cnh': 170, 'nad': 180, 'vid': 190, 'fic': 190, 'cont': 200, 'cpf': 210, 'recuper': 220, 'precis': 250, 'acess': 260, 'reconhec': 270, 'erro': 280, 'aplic': 290, 'funcion': 290, 'facial': 300, 'app': 350, 'tent': 370, 'cadastr': 410, 'consig': 430, 'faz': 560, 'senh': 570}


In [81]:
print(corpus['not_usability'])

{'horrivel': 7, 'app': 19, 'perd': 5, 'temp': 7, 'nao': 95, 'merec': 2, 'estrel': 5, 'dev': 5, 'ter': 6, 'pratic': 2, 'clarez': 1, 'funcion': 19, 'ped': 6, 'senh': 17, 'tempor': 1, 'envi': 1, 'pessim': 29, 'experienc': 5, 'nov': 3, 'plataform': 2, 'consig': 20, 'recuper': 7, 'acess': 15, 'nenhum': 4, 'opco': 2, 'fornec': 1, 'quer': 1, 'volt': 3, 'transtorn': 1, 'ser': 7, 'facil': 4, 'execut': 1, 'tend': 2, 'vist': 1, 'muit': 2, 'pesso': 4, 'nest': 2, 'moment': 1, 'dificuldad': 2, 'precis': 3, 'estar': 4, 'present': 1, 'virtual': 1, 'cad': 2, 'dia': 2, 'dificil': 4, 'lamentavel': 1, 'prov': 5, 'vid': 8, 'cam': 2, 'centraliz': 1, 'ate': 2, 'pra': 17, 'fech': 1, 'olhos': 1, 'hack': 1, 'desenvolv': 1, 'algo': 2, 'tao': 1, 'amador': 1, 'aplic': 19, 'nad': 6, 'nel': 2, 'faz': 25, 'cadastr': 23, 'cpf': 9, 'ja': 15, 'aparec': 2, 'telefon': 3, 'email': 2, 'sao': 1, 'porfavor': 1, 'soluca': 1, 'algu': 3, 'utiliz': 1, 'coloc': 9, 'pod': 7, 'coment': 1, 'referenc': 1, 'ordem': 1, 'mundial': 1, 'ch

In [26]:
remove_low_score(corpus)

In [28]:
{k: v for k, v in sorted(corpus['usability'].items(), key=lambda item: item[1])}


{'form': 10,
 'celul': 10,
 'gov': 10,
 'pois': 10,
 'nenhum': 11,
 'horrivel': 11,
 'vez': 11,
 'agor': 11,
 'consegu': 12,
 'email': 12,
 'pod': 13,
 'ajud': 13,
 'opca': 13,
 'ped': 13,
 'entrar': 14,
 'fiz': 14,
 'fot': 14,
 'ser': 15,
 'aparec': 15,
 'prov': 15,
 'fac': 15,
 'diz': 16,
 'temp': 17,
 'tod': 17,
 'cnh': 17,
 'nad': 18,
 'vid': 19,
 'fic': 19,
 'cont': 20,
 'nunc': 20,
 'pessim': 21,
 'cpf': 21,
 'recuper': 22,
 'precis': 25,
 'acess': 26,
 'reconhec': 27,
 'erro': 28,
 'aplic': 29,
 'funcion': 29,
 'facial': 30,
 'ja': 32,
 'app': 35,
 'tent': 37,
 'pra': 37,
 'cadastr': 41,
 'consig': 43,
 'faz': 56,
 'senh': 57,
 'nao': 183}

In [6]:
tmp_corpus = {
     'form': 10,
     'celul': 10,
     'vez': 11,
     'consegu': 12,
     'email': 12,
     'pod': 13,
     'ajud': 13,
     'opca': 13,
     'ped': 13,
     'entrar': 14,
     'fiz': 14,
     'fot': 14,
     'ser': 15,
     'aparec': 15,
     'prov': 15,
     'fac': 15,
     'diz': 16,
     'temp': 17,
     'tod': 17,
     'cnh': 17,
     'nad': 18,
     'vid': 19,
     'fic': 19,
     'cont': 20,
     'cpf': 21,
     'recuper': 22,
     'precis': 25,
     'acess': 26,
     'reconhec': 27,
     'erro': 28,
     'aplic': 29,
     'funcion': 29,
     'facial': 30,
     'app': 35,
     'tent': 37,
     'cadastr': 41,
     'consig': 43,
     'faz': 56,
     'senh': 57,
}

In [14]:
bla1 = {'form': 10,
 'celul': 10,
 'gov': 10,
 'pois': 10,
 'nenhum': 11,
 'horrivel': 11,
 'vez': 11,
 'agor': 11,
 'consegu': 12,
 'email': 12,
 'pod': 13,
 'ajud': 13,
 'opca': 13,
 'ped': 13,
 'entrar': 14,
 'fiz': 14,
 'fot': 14,
 'ser': 15,
 'aparec': 15,
 'prov': 15,
 'fac': 15,
 'diz': 16,
 'temp': 17,
 'tod': 17,
 'cnh': 17,
 'nad': 18,
 'vid': 19,
 'fic': 19,
 'cont': 20,
 'nunc': 20,
 'pessim': 21,
 'cpf': 21,
 'recuper': 22,
 'precis': 25,
 'acess': 26,
 'reconhec': 27,
 'erro': 28,
 'aplic': 29,
 'funcion': 29,
 'facial': 30,
 'ja': 32,
 'app': 35,
 'tent': 37,
 'pra': 37,
 'cadastr': 41,
 'consig': 43,
 'faz': 56,
 'senh': 57,
 'nao': 183}
bla2 = tmp_corpus
a = dict(set(bla1.items())-set(bla2.items()))
a

{'pois': 10,
 'agor': 11,
 'gov': 10,
 'pessim': 21,
 'nunc': 20,
 'pra': 37,
 'nenhum': 11,
 'nao': 183,
 'horrivel': 11,
 'ja': 32}